In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

def spearsonr_pval(x,y):
    return spearmanr(x,y)[1]

data_full = pd.read_csv("RA-extra-info-M-F-fix-08092022.csv")
data_a = data_full[data_full["Treatmentgroup"].isin(["Group1C","Group3E"])]

data_restore = pd.read_csv("RA-mass-restored-both.csv")
data_restore_list = data_restore.filename.to_list()
data_restore_list.insert(0, "MouseID")

# this is metadata info, which is extra baylor information.
# careful for the range, [a,b] mean a <= X < b. Also python is 0 based index.
data_M = data_a.iloc[:, np.r_[0, 2:9]]
data_M = data_M.set_index('MouseID')
# this is metabolites, which is features.
data_F = data_a[data_restore_list]
data_F = data_F.set_index('MouseID')

# get the two part of column names: metadata colum name and features column name.
M_column_list = data_M.head(0)
print(M_column_list)
F_column_list = data_F.head(0)
print(F_column_list)

df_col_list = []
df_corr_list = []
df_pval_list = []

for m_col in M_column_list:
    for f_col in F_column_list:
        col_1 = data_M[m_col]
        col_2 = data_F[f_col]
        corr_col_1_2 = col_1.corr(col_2, method='spearman')
        corr_pval_col_1_2 = col_1.corr(col_2, method=spearsonr_pval)
        df_col_list.append(m_col + "-" + f_col)
        df_corr_list.append(corr_col_1_2)
        df_pval_list.append(corr_pval_col_1_2)

print(df_col_list)
print(df_corr_list)
print(df_pval_list)

import statsmodels.api
dr_fdr_pval = statsmodels.stats.multitest.fdrcorrection(df_pval_list, method='indep')
df_final = pd.DataFrame(list(zip(df_col_list, df_corr_list, df_pval_list, dr_fdr_pval[0], dr_fdr_pval[1])), columns=['col_name','corr', 'corr_pval', 'FDR_rejected', 'FDR_pval'])

df_final.to_csv('RA-no-treat-both-corr-pval.csv', index=False)

Empty DataFrame
Columns: [Heart.Weight..Body.Weight, Liver.Weight..Body.Weight, Ejection.Fraction, PR.Interval..s., QTc..s., P.Amplitude..V., CD3.CD8.IFNg.]
Index: []
Empty DataFrame
Columns: [X105.07_2.767, X111.044_2.768, X141.959_6.859, X142.027_0.328, X192.125_4.643, X249.112_2.768, X263.091_2.771, X263.2_2.895, X276.02_2.431, X281.102_2.768, X285.133_2.768, X295.227_3.017, X296.258_2.976, X305.248_3.26, X369.17_2.767, X409.162_2.768, X425.136_2.773, X440.337_2.765, X448.342_2.867, X489.228_2.768, X538.315_2.977, X596.357_2.948, X650.978_2.339, X664.419_3.086, X668.414_2.975, X696.443_3.1, X748.439_2.944]
Index: []

[0 rows x 27 columns]
['Heart.Weight..Body.Weight-X105.07_2.767', 'Heart.Weight..Body.Weight-X111.044_2.768', 'Heart.Weight..Body.Weight-X141.959_6.859', 'Heart.Weight..Body.Weight-X142.027_0.328', 'Heart.Weight..Body.Weight-X192.125_4.643', 'Heart.Weight..Body.Weight-X249.112_2.768', 'Heart.Weight..Body.Weight-X263.091_2.771', 'Heart.Weight..Body.Weight-X263.2_2.895', 